<a href="https://colab.research.google.com/github/ammarwm/qantas/blob/main/Qantas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = '/content/conversion_rate.csv'

# Read the CSV file into a DataFrame
exchange_df = pd.read_csv(file_path, names=['from_currency', 'to_currency', 'rate'])

# Convert the 'rate' column to a float
exchange_df['rate'] = exchange_df['rate'].astype(float)

print('Before\n', exchange_df, '\n')

# Create a dictionary from the DataFrame
exchange_data = exchange_df.set_index('from_currency').to_dict(orient='index')

# Now, exchange_data contains the data in the desired dictionary structure
#print(exchange_data)

# Initialize an empty dictionary to store the updated exchange data. for indexing
updated_exchange_data = {}

# Iterate through the exchange_data dictionary
for from_currency, data in exchange_data.items():
    currency, to_currency, aud_rate = from_currency, data['to_currency'], data['rate']

    # Iterate through the conversion chain until reaching AUD
    while to_currency != 'AUD':
        currency, to_currency, rate = to_currency, exchange_data[to_currency]['to_currency'], exchange_data[to_currency]['rate']

        aud_rate *= rate

    # Update the updated_exchange_data dictionary
    updated_exchange_data.update({from_currency: {'to_currency': to_currency, 'rate': aud_rate}})

# Convert the updated exchange_data to a DataFrame
updated_exchange_df = pd.DataFrame.from_dict(updated_exchange_data, orient='index').reset_index()

# Rename the 'index' column to 'from_currency'
updated_exchange_df = updated_exchange_df.rename(columns={'index': 'from_currency'})

# Now, updated_exchange_df contains the data in the desired DataFrame structure
print('After\n', updated_exchange_df)

Before
   from_currency to_currency  rate
0           AUD         AUD   1.0
1            US          UK   2.0
2            UK          MY   3.0
3            MY         AUD   4.0 

After
   from_currency to_currency  rate
0           AUD         AUD   1.0
1            US         AUD  24.0
2            UK         AUD  12.0
3            MY         AUD   4.0


In [ ]:
# Load the CSV file containing values to be converted
values_df = pd.read_csv("values.csv", header=None, names=["id", "value", "currency"])

# Merge the two dataframes to get the conversion rates
merged_df = values_df.merge(updated_exchange_df, left_on="currency", right_on="from_currency")

merged_df['aud_value'] = merged_df['value'] * merged_df['rate']

merged_df

,id,value,currency,from_currency,to_currency,rate,aud_value
0,1,100,US,US,AUD,24.0,2400.0
1,2,200,AUD,AUD,AUD,1.0,200.0
2,3,1000,MY,MY,AUD,4.0,4000.0
